In [1]:
from db import *
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import py3Dmol
def draw_with_spheres(mol):
    v = py3Dmol.view(width=600,height=600)
    IPythonConsole.addMolToView(mol,v)
    v.zoomTo()
    v.setStyle({'sphere':{'radius':0.3},'stick':{'radius':0.2}})
    v.show()
IPythonConsole.ipython_3d = True

In [2]:
from typing import List
import asyncio
import aiohttp

async def get_material_name(
    calculate_name:str,
    cookie:str='JSESSIONID=840E07B635B0325B9A26E18FADC37F37'
)->Union[List[str],None]:
    url = 'http://192.168.0.238/sunera-dbsys/A0001_01/search'
    data = {
        'length': '50',
        'start': '0',
        'draw': '1',
        'jieGouMingCheng': calculate_name,
        'gongNengCeng': '',
        'leiXing': '',
        'caiLiaoMingCheng': '',
        'jiSuanZhuangTai': '',
        'smiles': '',
        'zhuanLiJieLun': '',
        'jieGouMingChengList': '',
        'search': 'true',
    }
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': cookie,
        'Origin': 'http://192.168.0.238',
        'Pragma': 'no-cache',
        'Proxy-Connection': 'keep-alive',
        'Referer': 'http://192.168.0.238/sunera-dbsys/A0001_01/init?init=true',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
    }
    try:
        async with aiohttp.ClientSession() as session:
            async with session.post(url, data=data, headers=headers,timeout=6) as response:
                data = await response.json()
                material_name = data['data'][0]['caiLiaoMingCheng']
                return [calculate_name, material_name] if material_name !='' else None
    except aiohttp.ClientError as e:
        print(f'HTTP请求异常: {e} {calculate_name}')
    except asyncio.TimeoutError:
        print('请求超时异常')
    except Exception as ex:
    	print(f'其他异常: {ex} {calculate_name}')


In [13]:
with open('name_pair2.txt','r') as f:
	lines = [line for line in f.readlines()]
	line=lines[1].strip().split(':')


['190123-CPO-0056', 'Z153']


In [1]:
import psycopg as pg
from psycopg.sql import SQL, Identifier, Placeholder

In [2]:
with pg.connect("dbname=emolecules user=postgres host=192.168.2.233 port=5432 connect_timeout=10 password=sunera") as conn:
	with conn.cursor() as cur:
		with open('name_pair2.txt','r') as f:
			lines = f.readlines()
			for line in lines:
				data=line.strip().split(':')
				cur.execute('UPDATE moldata.main SET materialname = %s WHERE structureName = %s',[data[1],data[0]])
print('over')

over


In [4]:
cal_names=[s[0] for s in raws]

In [5]:
pairs=[]
slice_cal_names=[]
for i in range(0,len(cal_names),200):
	try:
		slice_cal_names = cal_names[i:i+200]
	except IndexError:
		slice_cal_names = cal_names[i:]
	tasks = [asyncio.create_task(get_material_name(cal_name)) for cal_name in slice_cal_names]
	results = await asyncio.gather(*tasks)
	pair=[x for x in results if x is not None]
	pairs.extend(pair)
	print(i)

其他异常: list index out of range TC-18
其他异常: list index out of range TC-01-T3-1
其他异常: list index out of range TC-06
其他异常: list index out of range TC-204
其他异常: list index out of range TC-205
其他异常: list index out of range EBW-05
其他异常: list index out of range PXZ-TRZ
其他异常: list index out of range RD-603
其他异常: list index out of range TD-01
其他异常: list index out of range EBW-04
其他异常: list index out of range T3
其他异常: list index out of range BD-5A-2
其他异常: list index out of range DPQD
其他异常: list index out of range ACRXTN
其他异常: list index out of range BD-SYR-1
其他异常: list index out of range BD-SYR-3-2
其他异常: list index out of range EBK-06
其他异常: list index out of range TC-17
其他异常: list index out of range EBK-08
其他异常: list index out of range BD-SYR-2-4
其他异常: list index out of range EBK-09
其他异常: list index out of range TC-012
其他异常: list index out of range RD-607
其他异常: list index out of range TC-01
其他异常: list index out of range TC-10
其他异常: list index out of range SGR-A
其他异常: list index out of range SHF-D

In [6]:
await get_material_name('BD-B9')

其他异常: list index out of range BD-B9


In [6]:
with open('name_pair3.txt','w') as f:
	for pair in pairs:
		f.write(':'.join(pair)+'\n')

In [ ]:
select id,materialname,structurename,smiles from moldata.main where materialname is not null order by id desc limit 30;

In [ ]:
def add1(x):
    return x + 1

async def main_task(x):
    task1 = asyncio.create_task(add1(x))
    task2 = asyncio.create_task(add1(x+1))
    out = await asyncio.gather(task1, task2)
    return out

async def run_tasks():
	maintask1 = asyncio.create_task(main_task(1))
	maintask2 = asyncio.create_task(main_task(2))
	results = await asyncio.gather(maintask1, maintask2)
	return results

re=asyncio.run(run_tasks())
print(re)

In [ ]:
file_p=Path('Z:/Data-month-statistics/202111/1118/211109-CPM-0043/211109-CPM-0043.gjf/211109-CPM-0043.log')

In [ ]:
gaussian  	= GaussianOutput(file_p.as_posix())
xyz_str	=	gaussian.structures[-1].to(fmt="xyz", filename=None)
raw_mol = Chem.MolFromXYZBlock(xyz_str)
conn_mol = Chem.Mol(raw_mol) 
rdDetermineBonds.DetermineConnectivity(conn_mol)
draw_with_spheres(conn_mol)

In [ ]:
get_molData(file_p.as_posix())

In [ ]:
get_timestrp(file_p.as_posix())

In [ ]:
get_corrections(file_p.as_posix())

In [ ]:
gaussian = GaussianOutput(file_p.as_posix())
gaussian.final_energy

In [8]:
from typing import List,Union,Any
import asyncio
import aiohttp
import psycopg as pg
import nest_asyncio
nest_asyncio.apply()
async def get_material_name(calculate_name:str,cookie:str='JSESSIONID=DE46AE4688EA0CE5663FD26965FE050F')->Union[List[str],None]:
  url = 'http://192.168.0.238/sunera-dbsys/A0001_01search'
  data = {
    'length': '50',
    'start': '0',
    'draw': '1',
    'jieGouMingCheng': calculate_name,
    'gongNengCeng': '',
    'leiXing': '',
    'caiLiaoMingCheng': '',
    'jiSuanZhuangTai': '',
    'smiles': '',
    'zhuanLiJieLun': '',
    'jieGouMingChengList': '',
    'search': 'true',
  }
  headers = {
    'Accept': 'application/json, text/javascript,text/plain, */*; q=0.01',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': cookie,
    'Origin': 'http://192.168.0.238',
    'Pragma': 'no-cache',
    'Proxy-Connection': 'keep-alive',
    'Referer': 'http://192.168.0.238/sunera-dbsys/A0001_01/init?init=true',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
  }
  try:
    async with aiohttp.ClientSession() as session:
      async with session.post(url, data=data, headers=headers) as response:
        print('data:',response)
        data = await response.json()
        material_name = data['data'][0]['caiLiaoMingCheng']
        return [calculate_name, material_name] if material_name !='' else None
  except Exception as e:
    print('error',e)

async def main(cal_names:List[str])->List[str]:
	tasks = [asyncio.create_task(get_material_name(cal_name)) for cal_name in cal_names]
	results = await asyncio.gather(*tasks)
	print('sss',results)
	pair=[x for x in results if x is not None]
	return pair

slice_cal_names=['181213-HTL-0589','181229-CPO-0035','230109-GDF-0063']
name_pair=asyncio.run(main(slice_cal_names))
print(name_pair)

data: <ClientResponse(http://192.168.0.238/sunera-dbsys/A0001_01search) [500 Internal Server Error]>
<CIMultiDictProxy('Date': 'Sun, 23 Apr 2023 05:26:18 GMT', 'Server': 'Apache-Coyote/1.1', 'Content-Type': 'text/html;charset=utf-8', 'Content-Length': '1221', 'Connection': 'close')>

error 0, message='Attempt to decode JSON with unexpected mimetype: text/html;charset=utf-8', url=URL('http://192.168.0.238/sunera-dbsys/A0001_01search')
data: <ClientResponse(http://192.168.0.238/sunera-dbsys/A0001_01search) [500 Internal Server Error]>
<CIMultiDictProxy('Date': 'Sun, 23 Apr 2023 05:26:18 GMT', 'Server': 'Apache-Coyote/1.1', 'Content-Type': 'text/html;charset=utf-8', 'Content-Length': '1221', 'Connection': 'close')>

data: <ClientResponse(http://192.168.0.238/sunera-dbsys/A0001_01search) [500 Internal Server Error]>
<CIMultiDictProxy('Date': 'Sun, 23 Apr 2023 05:26:18 GMT', 'Server': 'Apache-Coyote/1.1', 'Content-Type': 'text/html;charset=utf-8', 'Content-Length': '1221', 'Connection': 'cl

In [9]:
import asyncio
import aiohttp

async def fetch(session, url):
    async with session.get(url) as response:
        if response.status != 200:
            print("Failed to fetch cookies")
            return None
        
        cookies = {}
        for key, value in response.cookies.items():
            cookies[key] = value.value
        return cookies

async def get_cookies(url):
    async with aiohttp.ClientSession() as session:
        cookies = await fetch(session, url)
        return cookies

if __name__ == "__main__":
    url = "http://192.168.0.238/sunera-dbsys/A0001_01search"
    loop = asyncio.get_event_loop()
    cookies = loop.run_until_complete(get_cookies(url))
    print(cookies)

{'JSESSIONID': '73375266E45FDC5D29FCC1779FF8D864'}


In [78]:
import aiohttp
import asyncio

async def fetch(session, url, headers, data):
	async with session.post(url, headers=headers, data=data) as response:
		return await response.json()

async def main():
	url = 'http://192.168.0.238/sunera-dbsys/A0001_01/search'
	headers = {
    'Accept': '*/*; q=0.01',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': 'JSESSIONID=E081D5A646855BA61B2E268C7F222472',
    'Origin': 'http://192.168.0.238',
    'Pragma': 'no-cache',
    'Proxy-Connection': 'keep-alive',
    'Referer': 'http://192.168.0.238/sunera-dbsys/A0001_01/init?init=true',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
	data = {
    'length': '50',
    'start': '0',
    'draw': '1',
    'jieGouMingCheng': '181213-HTL-0589',
    'gongNengCeng': '',
    'leiXing': '',
    'caiLiaoMingCheng': '',
    'jiSuanZhuangTai': '',
    'smiles': '',
    'zhuanLiJieLun': '',
    'jieGouMingChengList': '',
    'search': 'true'
	}

	async with aiohttp.ClientSession() as session:
		json_response = await fetch(session, url, headers, data)
		print(json_response)

loop = asyncio.get_event_loop()
s=loop.run_until_complete(main())

{'data': [{'caiLiaoMingCheng': 'B386', 'createBy': '殷梦轩', 'fenZiLiang': 728.28, 'imageObj': {'imageBase64String': 'iVBORw0KGgoAAAANSUhEUgAAAUsAAAEhCAYAAAD22IKiAAAq9klEQVR42u2dj6sdxfmHzx/nvYGAIBRspSDUCoVA20ClYhECWmkIDS0EBCFUAgFLQLzGEJG2icE0LQG5Vq3UIgiCIIIgBAShcL599uur28nsnrO7szPv7H4euGjur3Pu7uxn3t+z2QohhNjJRpdACCEklkIIIbEUQgiJpRBCSCzFnHz00Ufbw8PD7aOPPro9f/789uWXX97evXt3+/XXX+viCCGxFPDJJ580Innt2rXtBx980Ajl73//++3p06e3J0+ebL729NNPb1966aVGQD/77DNdNCEkluviiy++2D7yyCPbK1euRL+OZYnVeePGjcbiREC/973vbR9++OHtmTNnthcvXtz++c9/3n7++ee6mEJILJcJQoj4Pf/886OsUQT00qVLjdX50EMPNR8//elPty+++GIjoB9++KHceCGxFHXzn//8pxFKrEP+P5WV+vbbbzdiye/90Y9+tH3wwQcbAT137tz26OiosVIloEJiKaoBl/qXv/xlVChxsRG63/zmN0388r333tt+9dVXkwQUoXzuueea33twcLD98ssvdROExFL45oUXXtj+8Ic/7BQsLD9c6Ndee60R1Z/85CfbEydOJBNQ3HbcdCEklsItCB2ih8U3hJQCyu945plndDOExFL45Pbt200ShuRMCsYKKEJNLHOsZSqExFLMBnFDhJL/zklMQClNCiFeSjZdCImlcANF5NRG/u1vf3MVDsD6FEJiKVyAy012m+4bjwKuMiIhsRTFISZI/NCbUBrUX3qydoWQWK5UKIkLUtuYqug8NZcvX2560IWQWIpiUMtIeY5nN9dCBF7FXAiJ5cJhuAVZ6

In [49]:
import requests

url = 'http://192.168.0.238/sunera-dbsys/M0001_01/init'
headers = {
    'Accept': '*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache',
    'Proxy-Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers, verify=False)
cookie=response.headers['Set-Cookie'].split(";")[0]
print(cookie)

JSESSIONID=76386CAFB49A305C4174DD046DA05ABE


In [50]:
import requests

url = 'http://192.168.0.238/sunera-dbsys/M0001_01/regist'
headers = {
    'Accept': '*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': cookie,
    'Origin': 'http://192.168.0.238',
    'Pragma': 'no-cache',
    'Proxy-Connection': 'keep-alive',
    'Referer': 'http://192.168.0.238/sunera-dbsys/M0001_01/init',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}
data = {
    'regionUser': 'chensq',
    'password': 'Sunera@2013'
}

requests.post(url, headers=headers, data=data, verify=False)

<Response [200]>

In [88]:
import requests

url = 'http://192.168.0.238/sunera-dbsys/A0001_01/search'
headers = {
'Accept': '*/*; q=0.01',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cache-Control': 'no-cache',
'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
'Cookie': 'JSESSIONID=90E5E0D004BAC91406215A1E9D1C12C7',
'Origin': 'http://192.168.0.238',
'Pragma': 'no-cache',
'Proxy-Connection': 'keep-alive',
'Referer': 'http://192.168.0.238/sunera-dbsys/A0001_01/init?init=true',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}
data = {
'length': '50',
'start': '0',
'draw': '1',
'jieGouMingCheng': '181213-HTL-0589',
'gongNengCeng': '',
'leiXing': '',
'caiLiaoMingCheng': '',
'jiSuanZhuangTai': '',
'smiles': '',
'zhuanLiJieLun': '',
'jieGouMingChengList': '',
'search': 'true'
}

response = requests.post(url, headers=headers, data=data, verify=False)
json_data = response.content
print(json_data)

b'<script type="text/javascript" charset="UTF-8">parent.window.location.href=\'/sunera-dbsys/M0001_01/init\';</script>'


In [82]:
def get_cookie(username, password,url='http://192.168.0.238/sunera-dbsys/M0001_01'):
	import requests
	headers = {
		'Accept': '*/*;q=0.8',
		'Accept-Language': 'zh-CN,zh;q=0.9',
		'Cache-Control': 'no-cache',
		'Pragma': 'no-cache',
		'Proxy-Connection': 'keep-alive',
		'Upgrade-Insecure-Requests': '1',
		'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
	}
	response = requests.get(url + '/init', headers=headers, verify=False)
	cookie = response.headers['Set-Cookie'].split(";")[0]
	headers['Content-Type'] = 'application/x-www-form-urlencoded'
	headers['Cookie'] = cookie
	headers['Referer']= 'http://192.168.0.238/sunera-dbsys/A0001_01/init?init=true'
	data = {
	  'regionUser': username,
	  'password': password
	}
	requests.post(url + '/regist', headers=headers, data=data, verify=False)
	return cookie

In [83]:
get_cookie('chensq','Sunera@2013')

'JSESSIONID=90E5E0D004BAC91406215A1E9D1C12C7'

In [2]:
for n in range(200):
    i=2*n
    print('('+str(i+1)+')('+str(i+2)+')')

(1)(2)
(3)(4)
(5)(6)
(7)(8)
(9)(10)
(11)(12)
(13)(14)
(15)(16)
(17)(18)
(19)(20)
(21)(22)
(23)(24)
(25)(26)
(27)(28)
(29)(30)
(31)(32)
(33)(34)
(35)(36)
(37)(38)
(39)(40)
(41)(42)
(43)(44)
(45)(46)
(47)(48)
(49)(50)
(51)(52)
(53)(54)
(55)(56)
(57)(58)
(59)(60)
(61)(62)
(63)(64)
(65)(66)
(67)(68)
(69)(70)
(71)(72)
(73)(74)
(75)(76)
(77)(78)
(79)(80)
(81)(82)
(83)(84)
(85)(86)
(87)(88)
(89)(90)
(91)(92)
(93)(94)
(95)(96)
(97)(98)
(99)(100)
(101)(102)
(103)(104)
(105)(106)
(107)(108)
(109)(110)
(111)(112)
(113)(114)
(115)(116)
(117)(118)
(119)(120)
(121)(122)
(123)(124)
(125)(126)
(127)(128)
(129)(130)
(131)(132)
(133)(134)
(135)(136)
(137)(138)
(139)(140)
(141)(142)
(143)(144)
(145)(146)
(147)(148)
(149)(150)
(151)(152)
(153)(154)
(155)(156)
(157)(158)
(159)(160)
(161)(162)
(163)(164)
(165)(166)
(167)(168)
(169)(170)
(171)(172)
(173)(174)
(175)(176)
(177)(178)
(179)(180)
(181)(182)
(183)(184)
(185)(186)
(187)(188)
(189)(190)
(191)(192)
(193)(194)
(195)(196)
(197)(198)
(199)(200)
(201)(20